# Implement Conditional GAN on Accelerometer Data

In [53]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.data import DataLoader, ConcatDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Set random seed for reproducibility
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [26]:
DATA_DIR = f'{os.path.expanduser("~")}/.delta/nursing_pt'
import math
from torch.nn import functional as F
class WindowedDatasetWithStrideAndModeOfLabel(torch.utils.data.Dataset):
    def __init__(self, nurse, windowsize=1, stride=1):
        self.windowsize = windowsize
        self.stride = stride
        self.channels = 3
        self.X,self.y = torch.load(f'{DATA_DIR}/{nurse}.pt')
        # self.y = F.one_hot(self.y.long()).float()
        self.len = math.ceil(len(self.X)/self.stride)
        
        self.X = torch.cat([self.X,torch.zeros(self.windowsize-1,3)])

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return (
            self.X[(idx*self.stride):(idx*self.stride)+self.windowsize].transpose(0,1),
            self.y[(idx*self.stride):(idx*self.stride)+self.windowsize].mode().values
        )

def load_nursing_5_class(nurses='all', winsize=2001, test_size=0.25, batch_size=512, stride=1, split=None):
    not_labeled = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    unlabled_sessions = set.intersection(set(nurses), not_labeled)
    if unlabled_sessions:
        raise ValueError(f"Session indexes {unlabled_sessions} are not labled")

    if nurses == 'all':
        nurses = list(range(11, 71))

    if stride == 'partition':
        stride = winsize
    
    if test_size == 1:
        dev_idx = nurses
        devloader = DataLoader(dataset=ConcatDataset([WindowedDatasetWithStrideAndModeOfLabel(nurse=idx,windowsize=winsize,stride=stride) for idx in dev_idx]),batch_size=batch_size,shuffle=False)
        return None, devloader
    elif test_size == 0:
        train_idx = nurses
        trainloader = DataLoader(dataset=ConcatDataset([WindowedDatasetWithStrideAndModeOfLabel(nurse=idx,windowsize=winsize,stride=stride) for idx in train_idx]),batch_size=batch_size,shuffle=True)
        return trainloader, None
    
    if split:
        train_idx, dev_idx = split
        if set(train_idx).intersection(set(dev_idx)):
            raise ValueError(f"Train and dev indexes overlap")
        if set.intersection(set(train_idx).union(set(dev_idx)), not_labeled):
            raise ValueError(f"Some indexes are not labled")
    else:
        train_idx, dev_idx = train_test_split(nurses, test_size=test_size, random_state=0)
        print(dev_idx)
    trainloader = DataLoader(dataset=ConcatDataset([WindowedDatasetWithStrideAndModeOfLabel(nurse=idx,windowsize=winsize,stride=stride) for idx in train_idx]),batch_size=batch_size,shuffle=True)
    devloader = DataLoader(dataset=ConcatDataset([WindowedDatasetWithStrideAndModeOfLabel(nurse=idx,windowsize=winsize,stride=stride) for idx in dev_idx]),batch_size=batch_size,shuffle=False)
    
    return trainloader, devloader

In [134]:
workers = 2         # Number of workers for dataloader
batch_size = 128    # Batch size during training
window_size = 512   # Spatial length of data
nc = 3              # Number of channels
nz = 100            # Size of z latent vector (i.e. size of generator input)
ngf = 8             # Size of feature maps in generator
ndf = 4             # Size of feature maps in discriminator
num_epochs = 5      # Number of training epochs
lr = 0.0002         # Learning rate for optimizers
beta1 = 0.5         # Beta1 hyperparameter for Adam optimizers
ngpu = 2            # Number of GPUs available. Use 0 for CPU mode.
n_classes = 5       # Number of classes in the dataset
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Data

In [135]:
dataloader,_ = load_nursing_5_class(
    nurses='all',
    winsize=window_size,
    test_size=0,
    batch_size=batch_size,
    stride=window_size // 8
)

In [136]:
real_batch = next(iter(dataloader))
fig, axes = plt.subplots(3, 3, figsize=(15, 5))
plt.suptitle("Training Windows")
for i in range(9):
    axes.flatten()[i].plot(real_batch[0][i,0].numpy())
    axes.flatten()[i].plot(real_batch[0][i,1].numpy())
    axes.flatten()[i].plot(real_batch[0][i,2].numpy())
    axes.flatten()[i].axis("off")
plt.tight_layout()

# Models

In [137]:
# Weight initialization - call on model to initialize weights with N(0, 0.02)
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## Generator

In [138]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu

        self.label_emb = nn.Embedding(n_classes, n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(nz + n_classes, ngf, normalize=False),
            *block(ngf, ngf*2),
            *block(ngf*2, ngf*4),
            *block(ngf*4, ngf*8),
            nn.Linear(ngf*8, 3*window_size),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and signal to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        sig = self.model(gen_input)
        sig = sig.view(sig.size(0), 3, window_size)
        return sig

In [139]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
print(netG)

DataParallel(
  (module): Generator(
    (label_emb): Embedding(5, 5)
    (model): Sequential(
      (0): Linear(in_features=105, out_features=8, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Linear(in_features=8, out_features=16, bias=True)
      (3): BatchNorm1d(16, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
      (4): LeakyReLU(negative_slope=0.2, inplace=True)
      (5): Linear(in_features=16, out_features=32, bias=True)
      (6): BatchNorm1d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
      (7): LeakyReLU(negative_slope=0.2, inplace=True)
      (8): Linear(in_features=32, out_features=64, bias=True)
      (9): BatchNorm1d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
      (10): LeakyReLU(negative_slope=0.2, inplace=True)
      (11): Linear(in_features=64, out_features=1536, bias=True)
      (12): Tanh()
    )
  )
)


## Discriminator

In [140]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu

        self.label_embedding = nn.Embedding(n_classes, n_classes)

        self.model = nn.Sequential(
            nn.Linear(n_classes + 3*window_size, ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, ndf),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, ndf),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(ndf, 1),
        )

    def forward(self, sig, labels):
        # Concatenate label embedding and signal to produce input
        d_in = torch.cat((sig.view(sig.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

In [141]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the ``weights_init`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(weights_init)

# Print the model
print(netD)

DataParallel(
  (module): Discriminator(
    (label_embedding): Embedding(5, 5)
    (model): Sequential(
      (0): Linear(in_features=1541, out_features=4, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Linear(in_features=4, out_features=4, bias=True)
      (3): Dropout(p=0.4, inplace=False)
      (4): LeakyReLU(negative_slope=0.2, inplace=True)
      (5): Linear(in_features=4, out_features=4, bias=True)
      (6): Dropout(p=0.4, inplace=False)
      (7): LeakyReLU(negative_slope=0.2, inplace=True)
      (8): Linear(in_features=4, out_features=1, bias=True)
    )
  )
)


# Loss and Optimizers

In [142]:
adversarial_loss = torch.nn.MSELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, device=device)
fixed_gen_labels = torch.randint(0, 5, (64,), device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Train

In [143]:
from torch.autograd import Variable
from torch import FloatTensor, LongTensor
sig_list = []
G_losses = []
D_losses = []
iters = 0

netD.train()
netG.train()

num_epochs = 1000
for epoch in (pbar := tqdm(range(num_epochs))):
    # For each batch in the dataloader
    for i, (X,y) in enumerate(dataloader, 0):
        X,y = X.to(device), y.to(device)
        batch_size = X.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False).to(device)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False).to(device)

        # Configure input
        real_signals = Variable(X.type(FloatTensor))
        labels = Variable(y.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizerG.zero_grad()
        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, nz))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))
        # Generate a batch of signals
        gen_signals = netG(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = netD(gen_signals, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizerG.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizerD.zero_grad()

        # Loss for real images
        validity_real = netD(real_signals, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = netD(gen_signals.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizerD.step()

        G_losses.append(g_loss.item())
        D_losses.append(d_loss.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise, fixed_gen_labels).detach().cpu()
            sig_list.append(fake)
        iters += 1

    # if epoch % 10 == 0:
    # Output training stats
    # pbar.set_description(f'Loss_D: {errD.item():.4f}\tLoss_G: {errG.item():.4f}\tD(x): {D_x:.4f}\tD(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f}')
    pbar.set_description(f'Loss_D: {d_loss.item():.4f}\tLoss_G: {g_loss.item():.4f}\tD(x): {validity_real.mean().item():.4f}\tD(G(z)): {validity_fake.mean().item():.4f}')

Loss_D: 0.0789	Loss_G: 0.7365	D(x): 0.8523	D(G(z)): 0.1677:  43%|████▎     | 429/1000 [1:25:57<1:54:25, 12.02s/it]


KeyboardInterrupt: 

In [144]:
# Losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [146]:
# Show animation of sig_list

# plt.suptitle("Generated Windows")
fig = plt.figure(figsize=(8,8))
ims = [plt.plot(i[0].T, animated=True) for i in sig_list]
ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat_delay=1000)

# ims = []
# for sig in sig_list:
#     fig, axes = plt.subplots(3, 3, figsize=(15, 5))
#     for i in range(3):
#         for j in range(3):
#             axes[i,j].plot(sig[-1][i,0].numpy())
#             axes[i,j].plot(sig[-1][i,1].numpy())
#             axes[i,j].plot(sig[-1][i,2].numpy())
#             axes[i,j].axis("off")
#     ims.append(axes[i,j].lines)

HTML(ani.to_jshtml())

Animation size has reached 21102269 bytes, exceeding the limit of 20971520.0. If you're sure you want a larger animation embedded, set the animation.embed_limit rc parameter to a larger value (in MB). This and further frames will be dropped.


In [145]:
# Real Images vs. Fake Images
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))
fig, axes = plt.subplots(3, 6, figsize=(15, 5))

plt.suptitle("Real Images vs Generated Images")
# Plot the real images
for i in range(3):
    for j in range(3):
        axes[i,j].plot(real_batch[0][i+j,0].numpy())
        axes[i,j].plot(real_batch[0][i+j,1].numpy())
        axes[i,j].plot(real_batch[0][i+j,2].numpy())
        axes[i,j].axis("off")

# Plot the fake images from the last epoch
for i in range(3):
    for j in range(3,6):
        axes[i,j].plot(sig_list[-1][i+j,0].numpy())
        axes[i,j].plot(sig_list[-1][i+j,1].numpy())
        axes[i,j].plot(sig_list[-1][i+j,2].numpy())
        axes[i,j].axis("off")